In [19]:
import urllib.request
from geopandas import GeoDataFrame
import requests
import json
import numpy as np
import io
import pandas as pd
import choroplethNYC as cp
from fiona.crs import from_epsg
import shapely
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
import pandas as pd
import os

In [6]:
os.environ["PUIDATA"] = "%s/PUIdata"%os.getenv("HOME")
print("PUIdata", os.getenv("PUIDATA"))

PUIdata /nfshome/yf1150/PUIdata


In [20]:
pumashp = GeoDataFrame.from_file("%s/geo_export_b8d847b1-2542-460a-92c9-4dec773647ee.shp"%os.getenv("PUIDATA"))

In [21]:
pumashp.crs

{'init': 'epsg:4326'}

In [23]:
url = "https://api.census.gov/data/2016/acs/acs1/variables.json"
resp = requests.request('GET', url)
aff1y = json.loads(resp.text)

In [24]:
affkeys = np.array(list(aff1y['variables'].keys()))

In [25]:
[(k, aff1y['variables'][k]['label'])  for k in affkeys if k.startswith ("B28002") and 
 'Broadband' in aff1y['variables'][k]['label']]

[('B28002_007E',
  'Estimate!!Total!!With an Internet subscription!!Broadband such as cable, fiber optic or DSL'),
 ('B28002_004E',
  'Estimate!!Total!!With an Internet subscription!!Broadband of any type'),
 ('B28002_008E',
  'Estimate!!Total!!With an Internet subscription!!Broadband such as cable, fiber optic or DSL!!Broadband such as cable, fiber optic or DSL with no other type of Internet subscription')]

In [26]:
keyNhouseholds = 'B28002_001E'
aff1y['variables'][keyNhouseholds]

{'attributes': 'B28002_001M,B28002_001EA',
 'concept': 'PRESENCE AND TYPES OF INTERNET SUBSCRIPTIONS IN HOUSEHOLD',
 'group': 'B28002',
 'label': 'Estimate!!Total',
 'limit': 0,
 'predicateType': 'int'}

In [27]:
keyNBB = 'B28002_004E'
aff1y['variables'][keyNBB]

{'attributes': 'B28002_004M,B28002_004EA',
 'concept': 'PRESENCE AND TYPES OF INTERNET SUBSCRIPTIONS IN HOUSEHOLD',
 'group': 'B28002',
 'label': 'Estimate!!Total!!With an Internet subscription!!Broadband of any type',
 'limit': 0,
 'predicateType': 'int'}

In [28]:
myAPI = "43eee9769ca5327d47f53023ec549d59a850a874"

In [29]:
url = "https://api.census.gov/data/2016/acs/acs1?get=" + keyNBB +\
",NAME&for=public%20use%20microdata%20area:*&in=state:36&key=" + myAPI
resp = requests.request('GET', url).content
pumaBB = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')))

pumaBB.head()

,B28002_004E,NAME,state,public use microdata area,Unnamed: 4
0,32607,"St. Lawrence County PUMA, New York",36,100,NaN
1,50811,"Clinton, Franklin, Essex & Hamilton Counties P...",36,200,NaN
2,41210,"Warren & Washington Counties PUMA, New York",36,300,NaN
3,36075,Herkimer (North & Central) & Oneida (Outer) Co...,36,401,NaN
4,47786,Oneida County (Central)--Greater Utica & Rome ...,36,402,NaN


In [30]:
url = "https://api.census.gov/data/2016/acs/acs1?get=" + keyNhouseholds +\
",NAME&for=public%20use%20microdata%20area:*&in=state:36&key=" + myAPI
resp = requests.request('GET', url).content
pumaPP = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')))
pumaPP.head()

,B28002_001E,NAME,state,public use microdata area,Unnamed: 4
0,40479,"St. Lawrence County PUMA, New York",36,100,NaN
1,67092,"Clinton, Franklin, Essex & Hamilton Counties P...",36,200,NaN
2,51614,"Warren & Washington Counties PUMA, New York",36,300,NaN
3,44300,Herkimer (North & Central) & Oneida (Outer) Co...,36,401,NaN
4,61146,Oneida County (Central)--Greater Utica & Rome ...,36,402,NaN


In [31]:
pumaMerged = pumaBB.merge(pumaPP, on='NAME')

In [32]:
pumaMerged.drop(['Unnamed: 4_x', 'state_y', 'public use microdata area_y', 'Unnamed: 4_y'],axis=1, inplace=True)
pumaMerged.head(1)
pumaMerged.columns = ['B28002_004E', 'NAME', 'state', 'public use microdata area', 'B28002_001E']
pumaMerged.head(1)

,B28002_004E,NAME,state,public use microdata area,B28002_001E
0,32607,"St. Lawrence County PUMA, New York",36,100,40479


In [33]:
pumaMerged['percBB'] = pumaMerged['B28002_004E'] / pumaMerged['B28002_001E'] * 100
pumaMerged.head()

,B28002_004E,NAME,state,public use microdata area,B28002_001E,percBB
0,32607,"St. Lawrence County PUMA, New York",36,100,40479,80.552879
1,50811,"Clinton, Franklin, Essex & Hamilton Counties P...",36,200,67092,75.733321
2,41210,"Warren & Washington Counties PUMA, New York",36,300,51614,79.842678
3,36075,Herkimer (North & Central) & Oneida (Outer) Co...,36,401,44300,81.433409
4,47786,Oneida County (Central)--Greater Utica & Rome ...,36,402,61146,78.150656


In [17]:
bbpc = pd.read_csv(os.getenv("PUIDATA") + "/ACS_16_1YR_GCT2801.ST50_with_ann.csv",
            usecols=["GCT_STUB.target-geo-id2","HC01","HC02"])
bbpc.head()

,GCT_STUB.target-geo-id2,HC01,HC02
0,Target Geo Id2,Percent,Margin of Error
1,01,74.7,0.5
2,0100100,70.9,2.5
3,0100200,80.5,2.7
4,0100301,87.0,2.8


In [18]:
bbpc.drop([0,1], inplace=True)
bbpc.reset_index(inplace=True)
bbpc['GCT_STUB.target-geo-id2'] = bbpc['GCT_STUB.target-geo-id2'].astype(int)
bbpc['gid'] = bbpc['GCT_STUB.target-geo-id2'] - 3600000
bbpc.head()

,index,GCT_STUB.target-geo-id2,HC01,HC02,gid
0,2,100100,70.9,2.5,-3499900
1,3,100200,80.5,2.7,-3499800
2,4,100301,87.0,2.8,-3499699
3,5,100302,81.9,2.9,-3499698
4,6,100400,69.2,3.6,-3499600


In [9]:
# grab rows from 1 
bbpc = bbpc.iloc[1:,:]
# reset index starting from 0
bbpc.reset_index(inplace = True)
# take all rows from column 1
bbpc = bbpc.iloc[:,1:]

In [10]:
bbpc.head()

,GCT_STUB.target-geo-id2,HC01,HC02
0,01,74.7,0.5
1,0100100,70.9,2.5
2,0100200,80.5,2.7
3,0100301,87.0,2.8
4,0100302,81.9,2.9


In [11]:
# transform all things to float
for col in bbpc.columns[1:]:
    bbpc[col] = bbpc[col].astype('float')
# transform the target into int64
bbpc['GCT_STUB.target-geo-id2'] = bbpc['GCT_STUB.target-geo-id2']\
.str[-4:].astype('int64')
# rename it to gid and set inplace = true
bbpc.rename(columns={'GCT_STUB.target-geo-id2':'gid'}, \
            inplace=True)

In [13]:
bbpc.head(10)

,gid,HC01,HC02
0,1,74.7,0.5
1,100,70.9,2.5
2,200,80.5,2.7
3,301,87.0,2.8
4,302,81.9,2.9
5,400,69.2,3.6
6,500,81.3,2.8
7,600,68.6,2.6
8,700,70.3,3.1
9,800,74.7,3.4


In [34]:
diff_ = pumaMerged.merge(bbpc, right_on="gid", 
                     left_on="public use microdata area")[["percBB", "HC01"]]

diff_["diff"] = np.abs(diff_["percBB"] - diff_["HC01"].astype(float))

diff_.describe()

,percBB,diff
count,145.000000,145.000000
mean,81.845478,0.026217
std,7.063369,0.014480
min,54.932945,0.000178
25%,77.770257,0.012252
50%,82.826141,0.027230
75%,87.305077,0.039090
max,93.404143,0.049872
